In [1]:
!pip install pandas
!pip install nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# Loading data from CSV files and cleaning data
dogs = pd.read_csv("dogs.csv")
travels = pd.read_csv("dogTravel.csv", index_col=0)
NST_EST2021_POP = pd.read_csv("NST-EST2021-POP.csv", header=None, names=['State', 'Population'])
NST_EST2021_POP['Population'] = pd.to_numeric(NST_EST2021_POP['Population'].str.replace('.', ''))

# Shift the values of the specified columns to the next column and convert the "accessed" and "posted" columns to dates
cols_to_shift = ["status", "posted", "contact_city", "contact_state", "contact_zip", "contact_country", "stateQ","accessed"]
dogs.loc[dogs["status"] != "adoptable", cols_to_shift] = dogs.loc[dogs["status"] != "adoptable", cols_to_shift].shift(1, axis=1)
dogs['accessed'] = pd.to_datetime(dogs['accessed'], format='%Y-%m-%d').dt.date
dogs['posted'] = pd.to_datetime(dogs['posted'], format='%Y-%m-%dT%H:%M:%S%z').dt.date

C:\Users\feder\AppData\Local\Temp\ipykernel_18964\874685701.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NST_EST2021_POP['Population'] = pd.to_numeric(NST_EST2021_POP['Population'].str.replace('.', ''))


<h1> 1. Extract all dogs with status that is not adoptable

In [3]:
not_adoptable = dogs[dogs['status'] != 'adoptable']
print("Task 1: Dogs that are not adoptable")
print(not_adoptable['id'])

Task 1: Dogs that are not adoptable
644      41330726
5549     38169117
10888    45833989
11983    45515547
12495    45294115
12600    45229004
12613    45227052
17619    45569380
18611    44694387
19747    36978896
19845    33218331
22161    42092005
22229    39594038
29283    45895274
30471    45964719
31581    44538917
31888    41430442
33000    45907639
33527    45362806
34188    32590894
35065    31426754
44830    46037827
53168    44044071
53539    27521132
55434    38473806
55467    34101432
55915    45958435
55975    45927580
56013    45916348
56248    45733027
56464    45413997
56473    45406516
56541    45264615
Name: id, dtype: int64


<h1> 2. For each (primary) breed, determine the number of dogs

In [4]:
breed_counts = dogs['breed_primary'].value_counts()
print("Task 2: Number of dogs per primary breed")
print(breed_counts)

Task 2: Number of dogs per primary breed
Pit Bull Terrier                7890
Labrador Retriever              7198
Chihuahua                       3766
Mixed Breed                     3242
Terrier                         2641
                                ... 
Wirehaired Pointing Griffon        1
Boykin Spaniel                     1
Old English Sheepdog               1
Belgian Shepherd / Laekenois       1
Tosa Inu                           1
Name: breed_primary, Length: 216, dtype: int64


<h1> 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [5]:
breed_mixed_counts = dogs.groupby('breed_primary')['breed_mixed'].value_counts().unstack(fill_value=0)
breed_mixed_counts['Mixed/Non-Mixed Ratio'] = breed_mixed_counts[True] / breed_mixed_counts[False]
breed_mixed_counts = breed_mixed_counts.rename(columns={True: 'Mixed Breed', False: 'Non-Mixed Breed'})
print("Task 3: Ratio of Mixed Breed to Non-Mixed Breed for each primary breed")
print(breed_mixed_counts)


Task 3: Ratio of Mixed Breed to Non-Mixed Breed for each primary breed
breed_mixed                        Non-Mixed Breed  Mixed Breed  \
breed_primary                                                     
Affenpinscher                                   12            5   
Afghan Hound                                     0            4   
Airedale Terrier                                 2           17   
Akbash                                           1            2   
Akita                                           98           83   
...                                            ...          ...   
Wirehaired Pointing Griffon                      0            1   
Wirehaired Terrier                              15           45   
Xoloitzcuintli / Mexican Hairless                6            5   
Yellow Labrador Retriever                       36          122   
Yorkshire Terrier                              157          203   

breed_mixed                        Mixed/Non-Mixed Ratio 

<h1> 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [6]:
breed_posted_range = dogs.groupby('breed_primary')['posted'].agg(['min', 'max'])
print("Task 4: Posted timestamp range per primary breed")
print(breed_posted_range)

Task 4: Posted timestamp range per primary breed
                                          min         max
breed_primary                                            
Affenpinscher                      2012-03-08  2019-09-14
Afghan Hound                       2017-06-29  2019-07-27
Airedale Terrier                   2014-06-13  2019-09-19
Akbash                             2019-07-21  2019-08-23
Akita                              2012-03-03  2019-09-20
...                                       ...         ...
Wirehaired Pointing Griffon        2016-06-29  2016-06-29
Wirehaired Terrier                 2012-11-27  2019-09-19
Xoloitzcuintli / Mexican Hairless  2007-02-01  2019-09-08
Yellow Labrador Retriever          2010-05-31  2019-09-20
Yorkshire Terrier                  2012-02-04  2019-09-20

[216 rows x 2 columns]


<h1> 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [7]:
state_sex_counts = dogs.groupby(['contact_state', 'sex'])['id'].count().unstack(fill_value=0)
state_sex_counts['Male/Female Ratio'] = state_sex_counts['Male'] / state_sex_counts['Female']
state_sex_counts['Male-Female Delta'] = (state_sex_counts['Male'] - state_sex_counts['Female']).abs()
max_delta_state = state_sex_counts['Male-Female Delta'].idxmax()
max_ratio_state = state_sex_counts['Male/Female Ratio'].idxmax()
print("Task 5: Sex imbalance for each state")
print(state_sex_counts)
print(f"The state with the largest Male-Female Delta is {max_delta_state}")
print(f"The state with the largest Male/Female Ratio is {max_ratio_state}")

Task 5: Sex imbalance for each state
sex            Female  Male  Unknown  Male/Female Ratio  Male-Female Delta
contact_state                                                             
AK                  7     8        0           1.142857                  1
AL                716   712        0           0.994413                  4
AR                351   344        0           0.980057                  7
AZ               1067  1181        1           1.106842                114
CA                777   887        0           1.141570                110
CO                912   861        0           0.944079                 51
CT                682   740        0           1.085044                 58
DC                176   160        0           0.909091                 16
DE                148   148        0           1.000000                  0
FL               1279  1380        0           1.078968                101
GA               1727  1752        0           1.014476        

<h1> 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [8]:
age_size_stay_stats = dogs.groupby(['age', 'size'])[['stay_duration', 'stay_cost']].mean()
print("Task 6: Stay statistics per age and size")
print(age_size_stay_stats)

Task 6: Stay statistics per age and size
                    stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069


<h1> 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [9]:
dogs_travel_counts = travels.groupby('id')['contact_city'].count()
dogs_with_3_travels = dogs_travel_counts[dogs_travel_counts >= 3].index
dogs_with_3_travels_breeds = dogs[dogs['id'].isin(dogs_with_3_travels)][['id', 'breed_primary']]
print("Task 7: Dogs involved in at least 3 travels")
print(dogs_with_3_travels_breeds)

Task 7: Dogs involved in at least 3 travels
             id        breed_primary
1159   45642530                Jindo
6835   46039420        Border Collie
8526   40036107     Pit Bull Terrier
10681  45851842   Labrador Retriever
10803  45841145          Mixed Breed
...         ...                  ...
56850  41144335            Chihuahua
56864  40103682          Rat Terrier
56875  38664932     Pit Bull Terrier
56879  38495992     Pit Bull Terrier
57263  45588395  German Shepherd Dog

[563 rows x 2 columns]


<h1> 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [10]:
state_codes = {
    'Afghanistan': 'AF', 'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'Azerbaijan': 'AZ', 'Bahamas': 'BS', 'Bahrain': 'BH', 'California': 'CA', 'China': 'CN',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Egypt': 'EG', 'England': 'EN',
    'Florida': 'FL', 'Georgia': 'GA', 'Greece': 'GR', 'Hawaii': 'HI', 'Honduras': 'HO',
    'Idaho': 'ID', 'Illinois': 'IL', 'India': 'IND', 'Indiana': 'IN', 'Iowa': 'IA', 'Iran': 'IR',
    'Ireland': 'IR', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME',
    'Maryland': 'MD', 'Massachusetts': 'MA', 'Mexico': 'MX', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oman': 'OM',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Puerto Rico': 'PR', 'Qatar': 'QA',
    'Rhode Island': 'RI', 'Russia': 'RU', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'South Korea': 'KR', 'Spain': 'ES', 'St. Croix': 'SC', 'St. Maarten': 'SM', 'St. Simon': 'SI',
    'St. Thomas': 'ST', 'Taiwan': 'TW', 'Tennessee': 'TN', 'Texas': 'TX', 'Thailand': 'TH',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'Washington DC': 'DC',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}


travels['manual'] = travels['manual'].map(state_codes)
travels.loc[travels['manual'].notnull(), 'contact_state'] = travels['manual']
travels.loc[travels['manual'].notnull(), 'contact_city'] = travels['found']
print("Task 8: Correct state computed from manual and found fields")
print(travels[['id', 'contact_city', 'contact_state']])


Task 8: Correct state computed from manual and found fields
             id    contact_city contact_state
index                                        
0      44520267           Anoka            MN
1      44698509          Abacos            BS
2      45983838            Adam            MD
3      44475904     Saint Cloud            MN
4      43877389          Pueblo            CO
...         ...             ...           ...
6189   40492179        Fairmont            WV
6190   45799729  Eagle Mountain            UT
6191   34276515          Newnan            GA
6192   44519341           Young            OH
6193   36659999        New York            NY

[6194 rows x 3 columns]


<h1> 9. For each state, compute the ratio between the number of travels and the population.

In [11]:
# Adding a new column to the NST_EST2021_POP dataframe with the abbreviated state codes
NST_EST2021_POP['State Code'] = NST_EST2021_POP['State'].map(state_codes)

# Calculating the number of travels per state using the travels dataframe and the contact_state column as the grouping key.
travels_per_state = travels.groupby('contact_state').size().reset_index(name='Travels')

# Merging the travels_per_state and NST_EST2021_POP dataframes using the abbreviated state codes as the merge key.
travels_population = pd.merge(travels_per_state, NST_EST2021_POP, left_on='contact_state', right_on='State Code')

# Calculating the ratio between the number of travels and the population for each state.
travels_population['Travel/Population Ratio'] = travels_population['Travels'] / travels_population['Population']

# Printing the final result.
print("Task 9: Ratio between the number of travels and the population for each state")
print(travels_population[['State Code', 'Travel/Population Ratio']])


Task 9: Ratio between the number of travels and the population for each state
   State Code  Travel/Population Ratio
0          AL             1.612172e-05
1          AR             1.162202e-05
2          AZ             9.508492e-06
3          CA             4.223761e-06
4          CO             1.125792e-05
5          CT             2.024435e-05
6          DE             5.252801e-05
7          FL             7.567954e-06
8          GA             1.288286e-05
9          HI             1.374314e-06
10         IA             6.268867e-06
11         ID             5.437424e-07
12         IL             3.043901e-06
13         IN             8.842348e-06
14         KS             1.021144e-06
15         KY             1.176252e-05
16         LA             7.514346e-06
17         MA             1.152218e-05
18         MD             5.941180e-05
19         ME             1.181774e-04
20         MI             5.854725e-06
21         MN             2.488393e-05
22         MO            

<h1> 10. For each dog, compute the number of days from the posted day to the day of last access.

In [12]:
days_from_post_to_access = (dogs['accessed'] - dogs['posted']).dt.days
dogs['Days from Post to Access'] = days_from_post_to_access
print("Task 10: Number of days from the posted day to the day of last access for each dog")
print(dogs[['id', 'Days from Post to Access']])


Task 10: Number of days from the posted day to the day of last access for each dog
             id  Days from Post to Access
0      46042150                         0
1      46042002                         0
2      46040898                         0
3      46039877                         0
4      46039306                         0
...         ...                       ...
58175  44605893                       140
58176  44457061                       160
58177  42865848                       358
58178  42734734                       373
58179  42663515                       382

[58180 rows x 2 columns]


<h1> 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [13]:
weeks_from_post_to_access = (dogs['accessed'] - dogs['posted']).dt.days // 7
dogs['Weeks from Post to Access'] = weeks_from_post_to_access
dogs_partitioned = dogs.groupby('Weeks from Post to Access')['id'].apply(list)
print("Task 11: Dogs partitioned by the number of weeks from the posted day to the day of last access")
print(dogs_partitioned)

Task 11: Dogs partitioned by the number of weeks from the posted day to the day of last access
Weeks from Post to Access
0      [46042150, 46042002, 46040898, 46039877, 46039...
1      [45967088, 45966541, 45966538, 45966526, 45966...
2      [45892818, 45892807, 45892791, 45892773, 45892...
3      [45797757, 45797240, 45797231, 45791070, 45790...
4      [45727282, 45715803, 45710620, 45709771, 45705...
                             ...                        
729                                            [5142790]
746                                            [4527948]
811                                            [2613506]
812                                            [2592031]
852                                             [604115]
Name: id, Length: 579, dtype: object


<h1> 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [14]:
def common_word_percentage(row):
    # Tokenize the description
    description = row['description']
    if pd.isnull(description):
        return 0
    word_tokens = word_tokenize(description)

    # Calculate the percentage of common words
    common_words = set(word_tokens).intersection(row['group_description'])
    return 2 * len(common_words) / (len(word_tokens) + len(row['group_description']))

# Find the rows with exactly the same description using the duplicated() method
duplicates_100 = dogs[dogs.duplicated(subset=['sex', 'breed_primary', 'description'], keep=False)]

# Find the rows with less than 100% common words
remaining_rows = dogs.drop(duplicates_100.index)

# Group by 'sex' and 'breed_primary' and create a new column with the concatenated descriptions of each group
grouped = remaining_rows.groupby(['sex', 'breed_primary'])
remaining_rows['group_description'] = grouped['description'].transform(lambda x: ' '.join(x.dropna()))

# Create a new column with the percentage of common words
remaining_rows['common_word_percentage'] = remaining_rows.apply(common_word_percentage, axis=1)

# Filter the groups based on the condition that the common word percentage is greater than or equal to 0.9
duplicates_90 = grouped.filter(lambda x: x['common_word_percentage'].max() >= 0.9)

# Concatenate the two dataframes with 100% and 90% common words
duplicates = pd.concat([duplicates_100, duplicates_90])

# Drop the columns that were added for the calculation
duplicates = duplicates.drop(columns=['group_description', 'common_word_percentage'])
# Print the dataframe of duplicate records
print(duplicates['id'])


3        46039877
14       46038070
15       46038064
16       46038065
17       46038067
           ...   
58174    44629272
58175    44605893
58176    44457061
58178    42734734
58179    42663515
Name: id, Length: 11422, dtype: int64
